In [55]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from time import sleep
from random import randint

# API Camp to Camp - 

In [56]:
#Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        print(api_path)
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))

https://api.camptocamp.org/outings?act=skitouring&a=14274
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=30
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=60
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=90
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=120
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=150
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=180
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=210
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=240
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=270
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=300
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=330
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=360
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=390
https://api.camptocamp.org/outings?act=skitourin

https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3540
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3570
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3600
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3630
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3660
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3690
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3720
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3750
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3780
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3810
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3840
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3870
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3900
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=3930
https://api.camptoca

https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7080
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7110
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7140
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7170
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7200
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7230
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7260
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7290
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7320
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7350
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7380
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7410
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7440
https://api.camptocamp.org/outings?act=skitouring&a=14274&offset=7470
https://api.camptoca

https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1260
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1290
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1320
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1350
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1380
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1410
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1440
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1470
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1500
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1530
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1560
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1590
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=1620
https://api.camptocamp.org/outings?act=rock_climbin

https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4650
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4680
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4710
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4740
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4770
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4800
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4830
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4860
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4890
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4920
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4950
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=4980
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=5010
https://api.camptocamp.org/outings?act=rock_climbin

https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8040
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8070
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8100
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8130
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8160
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8190
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8220
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8250
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8280
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8310
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8340
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8370
https://api.camptocamp.org/outings?act=rock_climbing&a=14274&offset=8400
https://api.camptocamp.org/outings?act=rock_climbin

https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1470
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1500
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1530
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1560
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1590
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1620
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1650
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1680
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1710
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1740
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1770
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1800
https://api.camptocamp.org/outings?act=mountain_climbing&a=14274&offset=1830

https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1770
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1800
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1830
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1860
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1890
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1920
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1950
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=1980
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=2010
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=2040
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=2070
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=2100
https://api.camptocamp.org/outings?act=snow_ice_mixed&a=14274&offset=2130
https://api.camptocamp.org/outings?act

https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2190
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2220
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2250
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2280
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2310
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2340
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2370
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2400
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2430
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2460
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2490
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2520
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offset=2550
https://api.camptocamp.org/outings?act=ice_climbing&a=14274&offs

https://api.camptocamp.org/outings?act=hiking&a=14274&offset=2880
https://api.camptocamp.org/outings?act=hiking&a=14274&offset=2910
https://api.camptocamp.org/outings?act=hiking&a=14274&offset=2940
https://api.camptocamp.org/outings?act=hiking&a=14274&offset=2970
Nombre total de reqûetes :1040


# # Build dataframe & export to csv - 

In [67]:
# Flatten json file - 
d = pd.DataFrame()

for k in range(len(result)):    
    d=d.append(json_normalize(result[k]['documents'],max_level=1))

d.reset_index(inplace=True, drop=True)

# Select relevant columns -  
cols = ['document_id','activities','date_start','date_end','author.name','author.user_id']
d=d[cols]

# Print first rows
d.head()

,document_id,activities,date_start,date_end,author.name,author.user_id
0,1196222,[skitouring],2020-03-23,2020-03-23,christian73,1071965
1,1195244,"[skitouring, snow_ice_mixed, mountain_climbing...",2020-03-17,2020-03-17,dormeur10,800213
2,1195133,[skitouring],2020-03-17,2020-03-17,christian73,1071965
3,1195132,[skitouring],2020-03-17,2020-03-17,janic,578085
4,1195116,"[skitouring, snow_ice_mixed, paragliding]",2020-03-17,2020-03-17,Paulo,991164


In [68]:
# Clean columns -

# Clean activity - 
def dummies_activity(x,a):
    if a in x:
        return True
    else:
        return False

activities=['skitouring', 'rock_climbing','mountain_climbing','ice_climbing','hiking','snow_ice_mixed']

for activity in activities:
    d[activity]=d.activities.apply(lambda x: dummies_activity(x,activity))

d.drop('activities', inplace=True, axis=1)

# Drop_duplicates (caused by multi-activities outings exported several times during requests)
d.drop_duplicates(inplace=True)

# Convert to datetime
col_to_date=['date_end','date_start']
d[col_to_date]=d[col_to_date].apply(lambda x: pd.to_datetime(x))

# Convert to integer
col_to_int=['document_id','author.user_id']
d[col_to_int]=d[col_to_int].astype(int)

# Export to csv -
d.to_csv('outings.csv',index=False)

# CHECK - 
d.head()

,document_id,date_start,date_end,author.name,author.user_id,skitouring,rock_climbing,mountain_climbing,ice_climbing,hiking,snow_ice_mixed
0,1196222,2020-03-23,2020-03-23,christian73,1071965,True,False,False,False,False,False
1,1195244,2020-03-17,2020-03-17,dormeur10,800213,True,False,True,True,False,True
2,1195133,2020-03-17,2020-03-17,christian73,1071965,True,False,False,False,False,False
3,1195132,2020-03-17,2020-03-17,janic,578085,True,False,False,False,False,False
4,1195116,2020-03-17,2020-03-17,Paulo,991164,True,False,False,False,False,True
